Объявление нужных нам модулей и классов.

In [1]:
import os
from random import randint

import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

Создание DataFrame с именованием колонок.

In [ ]:
def append_abs_path(abs_path_to_files: list,
                    list_files: list, dir: list) -> None:
    '''Adding the full path to a file to the list'''
    for files in list_files:
        path = os.path.join(dir, files)
        abs_path_to_files.append(path)


def part_1() -> pd.core.series.Series:
    '''Creating a DataFrame with column names'''
    project_path = os.path.abspath("")
    cat_dir = os.path.join(project_path, "dataset", "cat")
    dog_dir = os.path.join(project_path, "dataset", "dog")

    files_cat = os.listdir(cat_dir)
    files_dog = os.listdir(dog_dir)
    class_ = list(["cat"] * len(files_cat)) + list(["dog"] * len(files_dog))

    abs_path_to_files = []
    append_abs_path(abs_path_to_files, files_cat, cat_dir)
    append_abs_path(abs_path_to_files, files_dog, dog_dir)

    return pd.DataFrame({
        "class": class_,
        "abspath": abs_path_to_files
    })

my_series = part_1()
my_series

Добавление колонки label, хранящая метку класса(0 для котов, 1 для собак).

In [ ]:
i = 0

for class_ in my_series["class"]:
    if class_ == "cat":
        my_series.loc[[i], ["label"]] = 0
    else:
        my_series.loc[[i], ["label"]] = 1
    i += 1

my_series

Добавление колонок с шириной, высотой и глубиной каждого изображения.

In [ ]:
relpaths = []
for i in range(0, my_series["abspath"].count()):
    abspath = my_series.iloc[i, 1]
    relpaths.append(abspath[-20:])
    
j = 0
for path in relpaths:
    image = cv2.imread(path)
    data = image.shape
    my_series.loc[[j], "height"] = data[0]
    my_series.loc[[j], "width"] = data[1]
    my_series.loc[[j], "depth"] = data[2]
    j += 1

my_series

Статистические данные о DataFrame(данные сбалансированы).

In [ ]:

my_series.describe()

Функция, возвращающая DataFrame, отфильтрованная по метке класса.

In [ ]:
def part_5(my_series: pd.core.series.Series,
           label: int) -> pd.core.series.Series:
    return my_series[my_series["label"] == label]

label_my_series = part_5(my_series, 1)
label_my_series

Функция, возвращающая отфильтрованный DataFrame по метке, высоте и ширине.

In [ ]:
def part_6(my_series: pd.core.series.Series, label: int,
           h: int, w: int) -> pd.core.series.Series:
    tmp_1 = my_series[my_series["label"] == label]
    tmp_2 = tmp_1[tmp_1["height"] <= h]
    return tmp_2[tmp_2["width"] <= w]

image_my_series = part_6(my_series, 0, 319, 500)
image_my_series

Функция, вычисляющая кол-во пикселей для исходного DataFrame и макс., мин. и ср. значения пикселей для DataFrame, отфильтрованного по метке.

In [ ]:
def part_7(label: int) -> None:
    my_series["pixels"] = (my_series["height"] *
                           my_series["width"]  *
                           my_series["depth"])

    pix_my_series = part_5(label)
    
    print("The maximum number of pixels is ", pix_my_series["pixels"].max())
    print("The minimum number of pixels is", pix_my_series["pixels"].min())
    print("The average number of pixels is", pix_my_series["pixels"].mean())

pixel_my_series = part_7(0)
pixel_my_series
my_series

Функция, строящая гистограмму.

In [79]:
def part_8(my_series: pd.core.series.Series, label: int) -> np.ndarray:
    label_series = my_series[my_series["label"] == label]
    rand_num = randint(0, label_series["abspath"].count())
    abspath = my_series.iloc[rand_num, 1]
    relpath = abspath[-20:]

    image = cv2.imread(relpath)

    histr_1 = cv2.calcHist([image], [0], None, [256], [0,256])
    histr_2 = cv2.calcHist([image], [1], None, [256], [0,256])
    histr_3 = cv2.calcHist([image], [2], None, [256], [0,256])
    
    return histr_1, histr_2, histr_3

Отрисовка гистограмм.

In [ ]:
histr_1, histr_2, histr_3 = part_8(my_series, 0)
col = ('b','g','r')
plt.plot(histr_1, color=col[0])
plt.plot(histr_2, color=col[1])
plt.plot(histr_3, color=col[2])
plt.xlabel("value")
plt.ylabel("count")
plt.title("histogram")
plt.xlim([0,256])
plt.show()